In [2]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.shl.com/products/product-catalog/"
start = 12
all_assessments = []

while True:
    url = f"{base_url}?start={start}&type=1&type=1"
    print("Scraping:", url)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all assessment cards
    cards = soup.select(".product-card")  # Adjust selector if needed

    if not cards:
        break  # No more pages

    for card in cards:
        item = {}

        # Assessment Name
        item["name"] = card.select_one(".product-card-title").get_text(strip=True)

        # URL to detail page
        item["url"] = card.select_one("a")["href"]

        # Remote support
        item["remote_support"] = card.select_one(".remote-testing").get_text(strip=True) if card.select_one(".remote-testing") else "Unknown"

        # Adaptive / IRT
        item["adaptive_support"] = card.select_one(".adaptive-testing").get_text(strip=True) if card.select_one(".adaptive-testing") else "Unknown"

        # Test type (P/K/etc.)
        item["test_type"] = card.select_one(".product-card-category").get_text(strip=True)

        all_assessments.append(item)

    start += 12

print("Total assessments scraped from listing pages:", len(all_assessments))


Scraping: https://www.shl.com/products/product-catalog/?start=12&type=1&type=1
Total assessments scraped from listing pages: 0


In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # remove this if you want to see the browser

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = webdriver.ChromeOptions()
# options.add_argument("--headless")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_data = []
start = 0

while True:
    if start == 0:
        url = "https://www.shl.com/products/product-catalog/"
    else:
        url = f"https://www.shl.com/products/product-catalog/?start={start}"

    print("\nScraping listing page:", url)
    driver.get(url)

    try:
        cards = WebDriverWait(driver, 12).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product"))
        )
    except Exception as e:
        print("No products found. Stopping.")
        break

    print("Found", len(cards), "products on this page.")

    if len(cards) == 0:
        break

    for card in cards:
        item = {}

        # Name
        try:
            item["name"] = card.find_element(By.CSS_SELECTOR, ".product__title").text
        except:
            item["name"] = ""

        # URL
        try:
            item["url"] = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        except:
            item["url"] = ""

        # Category (test type)
        try:
            item["test_type"] = card.find_element(By.CSS_SELECTOR, ".product__category").text
        except:
            item["test_type"] = ""

        # Remote testing tag
        try:
            item["remote_support"] = card.find_element(By.CSS_SELECTOR, ".product__tag--remote").text
        except:
            item["remote_support"] = "Unknown"

        # Adaptive tag
        try:
            item["adaptive_support"] = card.find_element(By.CSS_SELECTOR, ".product__tag--adaptive").text
        except:
            item["adaptive_support"] = "Unknown"

        all_data.append(item)

    start += 12



Scraping listing page: https://www.shl.com/products/product-catalog/
No products found. Stopping.


In [11]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()

# Make Selenium look human
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("start-maximized")

# Real user-agent
options.add_argument(
    "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# Remove navigator.webdriver (anti-bot)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

driver.get("https://www.shl.com/products/product-catalog/")
time.sleep(5)

print(driver.page_source[:5000])


<html lang="en-US" data-locale="en_US" data-language="en" data-localeroot="/" class="ss-productcataloguelistpage -colour-theme-blue || js-html-tag js" data-fl-id="1" data-geo="GB"><head>
        
<!-- Google Tag Manager -->
<script async="" src="https://scripts.clarity.ms/0.8.45/clarity.js"></script><script type="text/javascript" charset="UTF-8" async="" src="https://consent.cookiebot.com/34466d9c-a30f-4341-82cc-bfb2ce498814/cc.js?renew=false&amp;referer=www.shl.com&amp;dnt=false&amp;init=false"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=G-N151V2PPE2&amp;cx=c&amp;gtm=4e5ca1"></script><script async="" src="https://www.clarity.ms/tag/qgxubwj2gc?ref=gtm2"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-1009032174&amp;cx=c&amp;gtm=4e5ca1"></script><script type="text/javascript" async="" src="https://consent.cookiebot.com/uc.js?cbid=34466d9c-a30f-4341-82cc-bfb2ce4988

In [13]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.shl.com/products/product-catalog/")
time.sleep(5)

# Step 1: Find the CookieBot iFrame
iframe = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src*='cookiebot']"))
)
driver.switch_to.frame(iframe)

# Step 2: Click Accept (Allow all)
try:
    accept_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll")))
    accept_btn.click()
    print("Cookies accepted!")
except Exception as e:
    print("Cookie accept not found:", e)

# Switch back to main page
driver.switch_to.default_content()
time.sleep(5)

# NOW try to find shadow-root host
try:
    host = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "shl-products-catalog"))
    )
    shadow = driver.execute_script("return arguments[0].shadowRoot", host)
    cards = shadow.find_elements(By.CSS_SELECTOR, ".product")
    print("Products found:", len(cards))
except:
    print("Still no products found")


Cookie accept not found: Message: 
Stacktrace:
0   chromedriver                        0x0000000102fbc9d4 cxxbridge1$str$ptr + 3032992
1   chromedriver                        0x0000000102fb49c4 cxxbridge1$str$ptr + 3000208
2   chromedriver                        0x0000000102aaeb14 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74196
3   chromedriver                        0x0000000102af59c8 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 364680
4   chromedriver                        0x0000000102b36968 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 630824
5   chromedriver                        0x0000000102aea168 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 317480
6   chromedriver                        0x0000000102f8039c cxxbridge1$str$ptr + 2785640
7   chromedriver                        0x0000000102f83af8 cxxbridge1$str$ptr + 2799812
8   chromedriver                        0x0000000102f60b4c cxxbrid

In [26]:
from selenium import webdriver
import time 

driver = webdriver.Chrome()

driver.get("https://www.shl.com/products/product-catalog/")
time.sleep(2)
driver.find_element(By.CLASS_NAME, "CybotCookiebotDialogBodyButton").click()
time.sleep(2)
element = driver.find_element(By.CLASS_NAME, "custom__table-responsive")
print(element.text)
driver.quit()

Pre-packaged Job Solutions
Remote Testing
Adaptive/IRT
Test Type
Account Manager Solution
C
P
A
B
Administrative Professional - Short Form
A
K
P
Agency Manager Solution
A
B
P
S
Apprentice + 8.0 Job Focused Assessment
B
P
Apprentice 8.0 Job Focused Assessment
B
P
Bank Administrative Assistant - Short Form
A
B
K
P
Bank Collections Agent - Short Form
A
B
P
Bank Operations Supervisor - Short Form
A
B
P
S
Bilingual Spanish Reservation Agent Solution
B
P
S
A
Bookkeeping, Accounting, Auditing Clerk Short Form
P
S
K
B
A
Branch Manager - Short Form
A
B
P
Cashier Solution
B
A
P


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get("https://www.shl.com/products/product-catalog/")

# 1️⃣ Accept cookies (correct button)
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
    ).click()
except:
    pass  # If cookie popup doesn’t appear, ignore

# 2️⃣ Wait until the table becomes visible
table = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.custom__table-responsive table"))
)

# 3️⃣ Read rows
rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # skip header

scraped_data = []

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")

    # Name & link
    link_elem = cols[0].find_element(By.TAG_NAME, "a")
    name = link_elem.text.strip()
    link = link_elem.get_attribute("href")

    # Remote Testing
    remote = "Yes" if cols[1].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"

    # Adaptive / IRT
    adaptive = "Yes" if cols[2].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"

    # Test Types (A, B, C...)
    test_types = [span.text for span in cols[3].find_elements(By.CSS_SELECTOR, ".product-catalogue__key")]

    scraped_data.append([name, link, remote, adaptive, ",".join(test_types)])

# Print result
for item in scraped_data:
    print(item)

driver.quit()


['Account Manager Solution', 'https://www.shl.com/products/product-catalog/view/account-manager-solution/', 'Yes', 'Yes', 'C,P,A,B']
['Administrative Professional - Short Form', 'https://www.shl.com/products/product-catalog/view/administrative-professional-short-form/', 'Yes', 'Yes', 'A,K,P']
['Agency Manager Solution', 'https://www.shl.com/products/product-catalog/view/agency-manager-solution/', 'Yes', 'Yes', 'A,B,P,S']
['Apprentice + 8.0 Job Focused Assessment', 'https://www.shl.com/products/product-catalog/view/apprentice-8-0-job-focused-assessment-4261/', 'Yes', 'No', 'B,P']
['Apprentice 8.0 Job Focused Assessment', 'https://www.shl.com/products/product-catalog/view/apprentice-8-0-job-focused-assessment/', 'Yes', 'No', 'B,P']
['Bank Administrative Assistant - Short Form', 'https://www.shl.com/products/product-catalog/view/bank-administrative-assistant-short-form/', 'Yes', 'No', 'A,B,K,P']
['Bank Collections Agent - Short Form', 'https://www.shl.com/products/product-catalog/view/ban

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()

all_data = []

def scrape_page(start):
    url = f"https://www.shl.com/products/product-catalog/?start={start}"
    driver.get(url)

    # Accept cookies only on first page
    if start == 0:
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
            ).click()
        except:
            pass

    # Wait for table to load
    table = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.custom__table-responsive table"))
    )

    rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # skip header

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")

        # Name & link
        a = cols[0].find_element(By.TAG_NAME, "a")
        name = a.text.strip()
        link = a.get_attribute("href")

        # Remote
        remote = "Yes" if cols[1].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"

        # Adaptive
        adaptive = "Yes" if cols[2].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"

        # Test types
        test_types = [span.text for span in cols[3].find_elements(By.CSS_SELECTOR, ".product-catalogue__key")]

        all_data.append([
            name,
            link,
            remote,
            adaptive,
            ",".join(test_types)
        ])

# Loop through all pages
for start in range(0, 381, 12):   # 0 → 12 → 24 → ... → 372
    print(f"Scraping page: start={start}")
    scrape_page(start)
    time.sleep(2)

driver.quit()

# Print results
for row in all_data:
    print(row)

print("\nTotal items scraped:", len(all_data))


Scraping page: start=0
Scraping page: start=12
Scraping page: start=24
Scraping page: start=36
Scraping page: start=48
Scraping page: start=60
Scraping page: start=72
Scraping page: start=84
Scraping page: start=96
Scraping page: start=108
Scraping page: start=120
Scraping page: start=132
Scraping page: start=144
Scraping page: start=156
Scraping page: start=168
Scraping page: start=180
Scraping page: start=192
Scraping page: start=204
Scraping page: start=216
Scraping page: start=228
Scraping page: start=240
Scraping page: start=252
Scraping page: start=264
Scraping page: start=276
Scraping page: start=288
Scraping page: start=300
Scraping page: start=312
Scraping page: start=324
Scraping page: start=336
Scraping page: start=348
Scraping page: start=360
Scraping page: start=372
['Account Manager Solution', 'https://www.shl.com/products/product-catalog/view/account-manager-solution/', 'Yes', 'Yes', 'C,P,A,B']
['Administrative Professional - Short Form', 'https://www.shl.com/products/pr

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()

all_data = []

def scrape_page(start):
    url = f"https://www.shl.com/products/product-catalog/?start={start}"
    driver.get(url)

    # Accept cookies only once
    if start == 0:
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
            ).click()
        except:
            pass

        # Click All Products (VERY IMPORTANT)
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-filter='all']"))
            ).click()
            time.sleep(2)
        except:
            print("All Products button not clicked!")

    # Now wait for table
    table = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.custom__table-responsive table"))
    )

    rows = table.find_elements(By.TAG_NAME, "tr")[1:]

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")

        a = cols[0].find_element(By.TAG_NAME, "a")
        name = a.text.strip()
        link = a.get_attribute("href")

        remote = "Yes" if cols[1].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"
        adaptive = "Yes" if cols[2].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"

        test_types = [span.text for span in cols[3].find_elements(By.CSS_SELECTOR, ".product-catalogue__key")]

        all_data.append([name, link, remote, adaptive, ",".join(test_types)])


# Loop all pages
for start in range(0, 500, 12):  # large upper limit for safety
    print(f"Scraping page: start={start}")
    scrape_page(start)
    time.sleep(2)

driver.quit()

print("Total scraped:", len(all_data))


Scraping page: start=0
All Products button not clicked!
Scraping page: start=12
Scraping page: start=24
Scraping page: start=36
Scraping page: start=48
Scraping page: start=60
Scraping page: start=72
Scraping page: start=84
Scraping page: start=96
Scraping page: start=108
Scraping page: start=120
Scraping page: start=132
Scraping page: start=144
Scraping page: start=156
Scraping page: start=168
Scraping page: start=180
Scraping page: start=192
Scraping page: start=204
Scraping page: start=216
Scraping page: start=228
Scraping page: start=240
Scraping page: start=252
Scraping page: start=264
Scraping page: start=276
Scraping page: start=288
Scraping page: start=300
Scraping page: start=312
Scraping page: start=324
Scraping page: start=336
Scraping page: start=348
Scraping page: start=360
Scraping page: start=372
Scraping page: start=384
Scraping page: start=396
Scraping page: start=408
Scraping page: start=420
Scraping page: start=432
Scraping page: start=444
Scraping page: start=456
Sc

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
all_data = []

def scrape_table(table):
    rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # skip header

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")

        # name + link
        a = cols[0].find_element(By.TAG_NAME, "a")
        name = a.text.strip()
        link = a.get_attribute("href")

        # remote testing
        remote = "Yes" if cols[1].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"

        # adaptive
        adaptive = "Yes" if cols[2].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes") else "No"

        # test types
        test_types = ",".join([span.text for span in cols[3].find_elements(By.CSS_SELECTOR, ".product-catalogue__key")])

        all_data.append([name, link, remote, adaptive, test_types])


def scrape_page(start):
    url = f"https://www.shl.com/products/product-catalog/?type=1&start={start}"
    driver.get(url)

    # Accept cookies only on first page
    if start == 0:
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
            ).click()
            time.sleep(1)
        except:
            pass

    # Wait for ALL tables
    tables = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.custom__table-responsive table"))
    )

    # If start == 0 → pick **second** table
    if start == 0:
        print("Selecting 2nd table because start=0")
        target_table = tables[1]
    else:
        print("Selecting only table on the page")
        target_table = tables[0]

    scrape_table(target_table)


# Loop every page
for start in range(0, 500, 12):
    print(f"\nScraping page start={start}")
    scrape_page(start)
    time.sleep(1)

driver.quit()

print("\nTotal items scraped:", len(all_data))



Scraping page start=0
Selecting 2nd table because start=0

Scraping page start=12
Selecting only table on the page

Scraping page start=24
Selecting only table on the page

Scraping page start=36
Selecting only table on the page

Scraping page start=48
Selecting only table on the page

Scraping page start=60
Selecting only table on the page

Scraping page start=72
Selecting only table on the page

Scraping page start=84
Selecting only table on the page

Scraping page start=96
Selecting only table on the page

Scraping page start=108
Selecting only table on the page

Scraping page start=120
Selecting only table on the page

Scraping page start=132
Selecting only table on the page

Scraping page start=144
Selecting only table on the page

Scraping page start=156
Selecting only table on the page

Scraping page start=168
Selecting only table on the page

Scraping page start=180
Selecting only table on the page

Scraping page start=192
Selecting only table on the page

Scraping page start=

In [34]:
all_data[1][1]

'https://www.shl.com/products/product-catalog/view/net-framework-4-5/'

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# -------------------------------
# helper to extract text after <h4>
# -------------------------------

def get_text_after_heading(driver, heading):
    try:
        h4 = driver.find_element(By.XPATH, f"//h4[text()='{heading}']")
        p = h4.find_element(By.XPATH, "following-sibling::p[1]")
        return p.text.strip()
    except:
        return ""

# -------------------------------
# main enrichment process
# -------------------------------

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", False)
driver = webdriver.Chrome(options=options)

enhanced_data = []

for idx, row in enumerate(all_data):
    name, link, remote, adaptive, test_type = row

    print(f"({idx+1}/{len(all_data)}) Scraping → {name}")

    try:
        driver.get(link)
    except:
        # retry if driver lost connection
        driver.quit()
        driver = webdriver.Chrome(options=options)
        driver.get(link)

    # random sleep to avoid blocks
    time.sleep(random.uniform(1.0, 1.8))

    # Extract description + duration
    description = get_text_after_heading(driver, "Description")
    print(description)
    duration = get_text_after_heading(driver, "Assessment length")

    # Append new fields to row
    enhanced_data.append([
        name,
        link,
        remote,
        adaptive,
        test_type,
        description,
        duration
    ])

driver.quit()

print("\nDONE. Total enriched rows:", len(enhanced_data))


(1/377) Scraping → Global Skills Development Report
This report is designed to be given to individuals who have completed the Global Skills Assessment (GSA). With coverage across the Great 8 Domains, this measure of self-reported behaviors offers a complete overview of their current skills. Participants receive actionable tips on leveraging their top skill strengths and how they might develop their growth skills.
(2/377) Scraping → .NET Framework 4.5
The.NET Framework 4.5 test measures knowledge of .NET environment. Designed for experienced users, this test covers the following topics: Application Development, Application Foundation, Data Modeling, Deployment, Diagnostics, Performance, Portability, and Security.
(3/377) Scraping → .NET MVC (New)
Multi-choice test that measures the knowledge of Model-View-Controller (MVC) architecture, validation, security, routing, and areas.
(4/377) Scraping → .NET MVVM (New)
Multi-choice test that measures the knowledge of MVVM pattern, scenarios, da

In [39]:
enhanced_data

[['Global Skills Development Report',
  'https://www.shl.com/products/product-catalog/view/global-skills-development-report/',
  'Yes',
  'No',
  'A,E,B,C,D,P',
  'This report is designed to be given to individuals who have completed the Global Skills Assessment (GSA). With coverage across the Great 8 Domains, this measure of self-reported behaviors offers a complete overview of their current skills. Participants receive actionable tips on leveraging their top skill strengths and how they might develop their growth skills.',
  ''],
 ['.NET Framework 4.5',
  'https://www.shl.com/products/product-catalog/view/net-framework-4-5/',
  'Yes',
  'Yes',
  'K',
  'The.NET Framework 4.5 test measures knowledge of .NET environment. Designed for experienced users, this test covers the following topics: Application Development, Application Foundation, Data Modeling, Deployment, Diagnostics, Performance, Portability, and Security.',
  'Approximate Completion Time in minutes = 30'],
 ['.NET MVC (New)

In [40]:
import pandas as pd

columns = [
    'Assessment Name',
    'Link',
    'Remote Support (Yes/No)',
    'Adaptive/IRT (Yes/No)',
    'Test Type',
    'Description',
    'Duration'
]

df = pd.DataFrame(enhanced_data, columns=columns)
df.to_csv("SHLscrapedData.csv",index = False)